In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,170236.55,-2.27,0.84,-2.53,-0.07,-0.14,-0.01
4,-2.27,211863.23,-2.98,-2.35,2.15,0.27,0.14
5,0.84,-2.98,72602.36,0.35,0.88,0.73,-0.12
6,-2.53,-2.35,0.35,29838.99,-0.02,-0.02,-0.03
7,-0.07,2.15,0.88,-0.02,8458.35,0.06,0.04
8,-0.14,0.27,0.73,-0.02,0.06,1116.09,0.01
9,-0.01,0.14,-0.12,-0.03,0.04,0.01,107.69


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00001,0.00001,-0.00004,-0.00000,-0.00001,-0.00000
4,-0.00001,1.00000,-0.00002,-0.00003,0.00005,0.00002,0.00003
5,0.00001,-0.00002,1.00000,0.00001,0.00004,0.00008,-0.00004
6,-0.00004,-0.00003,0.00001,1.00000,-0.00000,-0.00000,-0.00002
7,-0.00000,0.00005,0.00004,-0.00000,1.00000,0.00002,0.00004
8,-0.00001,0.00002,0.00008,-0.00000,0.00002,1.00000,0.00002
9,-0.00000,0.00003,-0.00004,-0.00002,0.00004,0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

8.10735331468477

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.71307738e+09 3.90807237e+09 7.03182927e+08 1.10727684e+08
 7.89463134e+06 1.04280665e+05 1.29595474e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.311159,0.949253,-0.042843,-0.015648,-0.004334,-0.000571,-0.000054
1,0.944074,0.303210,-0.122535,-0.040603,-0.011099,-0.001462,-0.000141
2,0.097983,0.074276,0.983717,-0.127766,-0.029053,-0.003762,-0.000365
3,0.044894,0.035778,0.117640,0.985578,-0.106502,-0.012564,-0.001212
4,0.016677,0.013383,0.039378,0.100320,0.991557,-0.068572,-0.006441
5,0.003241,0.002602,0.007587,0.018499,0.065776,0.992926,-0.096714
6,0.000630,0.000505,0.001477,0.003593,0.012666,0.096024,0.995291


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.3111588307775448,
 0.6967901112813468,
 0.016282507024475312,
 0.014422377476123072,
 0.008443475394269817,
 0.007073586945231747,
 0.004709435133073758]